# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
file = os.path.join('..', 'output_data', 'cities.csv')
city_df = pd.read_csv(file)
city_df.head()

,City,Country,Latitude,Longitude,Humidity,Wind Speed,Cloudiness,Temperature,Date
0,Longyearbyen,SJ,78.2186,15.6401,81,3.44,75,39.04,1655040711
1,Rikitea,PF,-23.1203,-134.9692,63,8.88,67,72.82,1655040712
2,Tuktoyaktuk,CA,69.4541,-133.0374,100,9.22,100,32.00,1655040713
3,Hong Kong,HK,22.2855,114.1577,88,11.01,100,84.38,1655040593
4,Butaritari,KI,3.0707,172.7902,75,12.93,15,81.43,1655040714


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

In [12]:
locations = city_df[['Latitude', 'Longitude']]
humidity = city_df['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=30, point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
ideal_weather = city_df[(city_df['Temperature'] >= 70) & (city_df['Temperature'] <= 80)]
ideal_weather = ideal_weather[ideal_weather['Wind Speed'] <= 10]
ideal_weather = ideal_weather[ideal_weather['Cloudiness'] == 0]

ideal_weather.head()

,City,Country,Latitude,Longitude,Humidity,Wind Speed,Cloudiness,Temperature,Date
207,Arrondissement du Havre,FR,49.5833,0.3333,72,9.22,0,70.34,1655040888
253,Seoul,KR,37.5683,126.9778,71,5.75,0,74.59,1655040925
304,Jiuquan,CN,39.7432,98.5174,9,1.52,0,71.91,1655040967
335,Chiredzi,ZW,-21.0500,31.6667,32,3.29,0,73.87,1655040993
350,Plymouth,US,41.8334,-70.8328,67,9.22,0,75.06,1655040519


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Country,Latitude,Longitude,Humidity,Wind Speed,Cloudiness,Temperature,Date,Hotel Name
207,Arrondissement du Havre,FR,49.5833,0.3333,72,9.22,0,70.34,1655040888,
253,Seoul,KR,37.5683,126.9778,71,5.75,0,74.59,1655040925,
304,Jiuquan,CN,39.7432,98.5174,9,1.52,0,71.91,1655040967,
335,Chiredzi,ZW,-21.0500,31.6667,32,3.29,0,73.87,1655040993,
350,Plymouth,US,41.8334,-70.8328,67,9.22,0,75.06,1655040519,


In [20]:
for index, row in hotel_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "keyword": "hotel",
            "radius" : 5000,
            "key": g_key
        }
        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat},{lng}"
        hotel_response = requests.get(url, params=params).json()

        hotel_df.loc[index, "Hotel Name"] = hotel_response['results'][0]['name']

    except:
        hotel_df.loc[index, "Hotel Name"] = "NaN"

hotel_df.head()

,City,Country,Latitude,Longitude,Humidity,Wind Speed,Cloudiness,Temperature,Date,Hotel Name
207,Arrondissement du Havre,FR,49.5833,0.3333,72,9.22,0,70.34,1655040888,Manoir de l'oiselière
253,Seoul,KR,37.5683,126.9778,71,5.75,0,74.59,1655040925,"Four Points by Sheraton Josun, Seoul Station"
304,Jiuquan,CN,39.7432,98.5174,9,1.52,0,71.91,1655040967,Jiuquan Hotel
335,Chiredzi,ZW,-21.0500,31.6667,32,3.29,0,73.87,1655040993,Clevers Hotel Chiredzi
350,Plymouth,US,41.8334,-70.8328,67,9.22,0,75.06,1655040519,Rodeway Inn Middleboro-Plymouth


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))